# Homework 6 

## Section 1: Introduction

In [1]:
#Load Packages

from sklearn import metrics
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd


In [2]:
#Import dataset

train = pd.read_csv('/Users/anjali/Documents/SPRING22/IST 707/Week6/digit-train.csv')
test = pd.read_csv('/Users/anjali/Documents/SPRING22/IST 707/Week6/digit-test.csv')


In [3]:
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
test.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Data Pre-processing

The target variable in the train dataset, stored as colum 'label' is subsetted and stored in y_train while the other attributes are stored in X_train. 

In [5]:
y_train = train["label"]

# Drop 'label' column
X_train = train.drop(labels = ["label"],axis = 1) 


y_train.value_counts()


1    471
8    438
7    436
3    425
4    420
0    418
6    416
2    413
5    390
9    371
Name: label, dtype: int64

In [6]:
#Same is done for test dataset for later use

y_test = test["label"]

# Drop 'label' column
X_test = test.drop(labels = ["label"],axis = 1)


In [7]:
#Check for missing values

y_train.isna().sum()

0

There are no missing values.

# Section 2: Decision tree

First the original decision tree model is create and trained using the whole training dataset and then 3-fold cross validation is performed for training the model and the difference in of the 2 models is compared.

In [8]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

#Creating a pipeline that consists of steps : Scaling and Decision tree classifier

pipeline = make_pipeline(StandardScaler(), DecisionTreeClassifier(random_state=1046,max_depth = 12,
                                                                  min_samples_leaf=2))


pipeline = make_pipeline(StandardScaler(), DecisionTreeClassifier(random_state=1046,max_depth = 12,
                                                                  min_samples_leaf=2))

#The max_depth parameter in the model defines the maximum allowed size to which the decision tree is allowed to expand 
#while, the min_samples_leaf paramter defines the minimum sample leaves needed in the decision tree

In [9]:
pipeline.fit(X_train,y_train)
predict= pipeline.predict(X_train)

trainaccuracy= accuracy_score(y_train,pipeline.predict(X_train))
trainaccuracy
print("Train Data Accuracy    :{} %".format(round((trainaccuracy*100),2)))



Train Data Accuracy    :95.31 %


The original Decision tree model gives a training set accuracy of 95.31%. Such high accuracy can lead to  overfitting.

### Cross-validation

K-fold cross validation is a data splitting technique that can be implemented with k > 1 folds.
The conventional technique for training and testing the model is to split the data in two different splits which are termed as training and test split but this technique has it's disadvantages as models tend to overfit to test data and cannot perform for un-seen data.To overcome this cross-validation is used that creates K-folds of training data in which (K-1) fold is used for training and the remaining fold is used for testing. This process is repeated for K times and the model performance is calculated for a particular set of hyperparameters by taking the mean and standard deviation of all the K models created.

In [10]:
#Cross-validation
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

#Tune the parameters, such as the pruning options, and report the 3-fold CV accuracy. 
cv =  KFold(n_splits=3, random_state=1, shuffle=True)

# evaluate model
scores = cross_val_score(pipeline, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)

In [11]:
print("Cross-validation Training accuracy scores %s" %scores)

print("Cross-validation Training accuracy: %.3f +/- %.3f" %(np.mean(scores)*100,np.std(scores)*100))

Cross-validation Training accuracy scores [0.74142857 0.73909936 0.74267334]
Cross-validation Training accuracy: 74.107 +/- 0.148


In [12]:
#predict 

#Now predicting the digits from the test dataset and computing the performance of the model with test dataset

result = pipeline.predict(X_test)

testaccuracy = accuracy_score(y_test,result)
testaccuracy
print("Test Data Accuracy    :{} %".format(round((testaccuracy*100),2)))

Test Data Accuracy    :77.58 %


The accuracy of the Decision tree model with 3-fold cross validation is 77.585%.

# Section 3: Naïve Bayes

Naive Bayes is a classification technique based on the Bayes theorem. It is a simple but powerful algorithm for predictive modeling under supervised learning algorithms.The specific Naive Bayes model we use here is the Gaussian Naive Bayes. This is a variant of Naive Bayes which supports continuous values and has an assumption that each class is normally distributed. 

In [13]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

#Create a pipeline which first performs PCA on the train dataset followed by creating the GaussianNB()
pipe = Pipeline(steps=[('pca', PCA()),('estimator', GaussianNB()),])
    
pipe.fit(X_train, y_train)

trainaccuracy = accuracy_score(y_train,pipe.predict(X_train))
print("Train Data Accuracy    :{} %".format(round((trainaccuracy*100),2)))



Train Data Accuracy    :48.69 %


In [14]:
#Cross-validation

scores = cross_val_score(pipe, X_train, y_train, cv = 3, scoring='accuracy')


The original model accuracy on taining dataset was 48.69% and accuracy after performing 3-fold cross validation is 52.073%. So, the 3-fold cross-validation accuracy does not result in performance improvement for this model.

### Hypertuning parameters

In [15]:
parameters = {'estimator__var_smoothing': [1e-11, 1e-10, 1e-9]}
Bayes = GridSearchCV(pipe, parameters, scoring='accuracy', cv=3).fit(X_train, y_train)
print(Bayes.best_estimator_)
print('best score:')
print(Bayes.best_score_)
predictions = Bayes.best_estimator_.predict(X_test)
print(predictions)

Pipeline(steps=[('pca', PCA()), ('estimator', GaussianNB())])
best score:
0.5207284114503556
[0 8 9 ... 5 0 1]


The accuracy of the Naive Bayes model with 3-fold cross validation is 52.072%

The Naive Bayes model yields an accuracy of approximately 52% on the given test set while Decision tree gives an accuracy of 77.6%. Clearly, Decision tree model performs better than Naive Bayes model.
Additionally, the Decision tree model runs faster than the Naive Bayes model.

Decision tree is a discriminative model, whereas Naive bayes is a generative model. Decision trees are more flexible and easy and perform better when there is lots of data. Naive Bayes on the other hand performs better with less data.This is the reason Naive Bayes is slower.